# Functions

1. encode_text_features
1. split_dataset
1. scale_train_dataset
1. scale_train_dataset
1. perform_dataset_ops
1. save_learned_models
1. load_learned_models

In [5]:
def save_dict_to_csv(dct,fname,name='algos'):
  fileName =  drive_location+fname+".csv"
  header = sorted(set(i for b in map(dict.keys, dct.values()) for i in b))
  with open(fileName, 'w', newline="") as f:
    write = csv.writer(f)
    write.writerow([name, *header])
    for a, b in dct.items():
      write.writerow([a]+[b.get(i, '') for i in header])

In [6]:
def encode_text_features(encode_decode, data_frame, encoder_isa=None, encoder_mem_type=None):
  # Implement Categorical OneHot encoding for ISA and mem-type
  if encode_decode == 'encode':
    encoder_isa = ce.one_hot.OneHotEncoder(cols=['isa'])
    encoder_mem_type = ce.one_hot.OneHotEncoder(cols=['mem_type'])
    encoder_isa.fit(data_frame, verbose=1)
    df_new1 = encoder_isa.transform(data_frame)
    encoder_mem_type.fit(df_new1, verbose=1)
    df_new = encoder_mem_type.transform(df_new1)
    encoded_data_frame = df_new
  else:
    df_new1 = encoder_isa.transform(data_frame)
    df_new = encoder_mem_type.transform(df_new1)
    encoded_data_frame = df_new
        
  return encoded_data_frame, encoder_isa, encoder_mem_type

In [7]:
def split_dataset(X, y):
  # split into train and test
  #X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, X.index, test_size=0.2, random_state=42)
  # use default random state of np.random, so not providing any value here
  X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, X.index, test_size=0.2, random_state=0)
  return X_train, X_test, y_train, y_test, indices_train, indices_test

In [8]:
def scale_train_dataset(X_train, y_train, input_scaler=None, output_scaler=None):
  # scale X_train input (fit and transform)
  if input_scaler is not None:
    # fit scaler on training dataset
    input_scaler.fit(X_train)
    # transform training dataset
    X_train = input_scaler.transform(X_train)        
        
  # scale y_train input (fit and transform)
  if output_scaler is not None:
    # fit scaler on training dataset
    output_scaler.fit(y_train)
    # transform training dataset
    y_train = output_scaler.transform(y_train)        
        
  return X_train, y_train, input_scaler, output_scaler

In [9]:
def scale_test_dataset(X_test, y_test, input_scaler, output_scaler):
  # scale X_test input (only transform)
  if input_scaler is not None:
    # transform test dataset
    X_test = input_scaler.transform(X_test)
        
  # scale y_train input (only transform)
  if output_scaler is not None:
    # transform test dataset
    y_test = output_scaler.transform(y_test)
        
  return X_test, y_test

In [10]:
def perform_dataset_ops(benchmark, algo, algo_fname):
  print(benchmark, algo, algo_fname)
  df_orig=pd.read_csv(drive_location+algo_fname+".csv")
  #df_orig.dropna(inplace=True)
    
  # Encode text features
  df_encoded, encoder_isa, encoder_mem_type = encode_text_features('encode', df_orig)
  print("df_orig====================>",df_orig)
  print("df_encoded====================>",df_encoded)
  # store learned models in dictory
  dataset = {}
  dataset['encoder_isa'] = encoder_isa
  dataset['encoder_mem_type'] = encoder_mem_type
    
  dataset['df_orig'] = df_orig
  dataset['df_encoded'] = df_encoded

#  for ps_indx, ps in enumerate(df_orig.PS.unique()):
#    print(ps_indx, ps)
#    dataset[ps] = {}
    
#    criteria_df = (df_encoded['PS'] == ps)
    #print(criteria_df)
#    df_selected = df_encoded[criteria_df]
  df_selected = df_encoded    
  if (benchmark == "SPEC2006" or benchmark == "SPEC2017"):
    X_all=df_selected.drop(columns=['arch','ld_shared_by_cores','l2_shared_by_cores','no_of_threads','system_name','bus_speed_qpi','bus_speed_dmi','l1_ins_assoc','l1_data_assoc','l2_assoc','l3_assoc','raw_bus_speed','converted_bus_speed','ddr_type','runtime'])
  elif (benchmark == "NPB"):
    X_all=df_selected.drop(columns=['sys','sysname','arch','l1d_assoc','l1d_cache_lines','l1d_shared_by_threads','l2_assoc','l2_cache_lines','l2_shared_by_threads','l3_assoc','l3_cache_lines','l3_shared_by_threads','PS','runtime','executable','system'])
  else:
    X_all=df_selected.drop(columns=['sys','arch','l1d_assoc','l1d_cache_lines','l1d_shared_by_threads','l2_assoc','l2_cache_lines','l2_shared_by_threads','l3_assoc','l3_cache_lines','l3_shared_by_threads','runtime'])
  #print(X_all)
  X_all.dropna(inplace=True)
  dataset['X_all'] = X_all
  
  # Output
  y=df_selected[['runtime']]
  print("y shape ",y.shape, "y types", y.dtypes)
  #print("y shape ",y.shape, "y types", y.dtypes)
  dataset['y'] = y

  # split into train and test
  X_train, X_test, y_train, y_test, indices_train, indices_test = split_dataset(X_all, y)
  dataset['X_train'] = X_train
  dataset['X_test'] = X_test
  dataset['y_train'] = y_train
  dataset['y_test'] = y_test
  dataset['indices_train'] = indices_train
  dataset['indices_test'] = indices_test

  # unscaled inputs
  # scale train data (fit and transform)
  X_train_unscaled, y_train_unscaled, input_scaler_unscaled, output_scaler_unscaled = scale_train_dataset(X_train, y_train, None, StandardScaler())
  # scale test data (only transform)
  X_test_unscaled, y_test_unscaled = scale_test_dataset(X_test, y_test, input_scaler_unscaled, output_scaler_unscaled)
  dataset['X_train_unscaled'] = X_train_unscaled
  dataset['X_test_unscaled'] = X_test_unscaled
  dataset['y_train_unscaled'] = y_train_unscaled
  dataset['y_test_unscaled'] = y_test_unscaled
  dataset['input_scaler_unscaled'] = input_scaler_unscaled
  dataset['output_scaler_unscaled'] = output_scaler_unscaled
      
  #standardized inputs
  # scale train data (fit and transform)
  X_train_standardized, y_train_standardized, input_scaler_standardized, output_scaler_standardized = scale_train_dataset(X_train, y_train, StandardScaler(), StandardScaler())
  # scale test data (only transform)
  X_test_standardized, y_test_standardized = scale_test_dataset(X_test, y_test, input_scaler_standardized, output_scaler_standardized)
  dataset['X_train_standardized'] = X_train_standardized
  dataset['X_test_standardized'] = X_test_standardized
  dataset['y_train_standardized'] = y_train_standardized
  dataset['y_test_standardized'] = y_test_standardized
  dataset['input_scaler_standardized'] = input_scaler_standardized
  dataset['output_scaler_standardized'] = output_scaler_standardized
      
  save_learned_models(dataset, algo, "dataset")

In [11]:
def save_learned_models(dct, algo, fname):
  pickleFile = drive_location+algo+"_"+fname+".pickle"
  with open(pickleFile, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(dct, f, pickle.HIGHEST_PROTOCOL)

In [12]:
def load_learned_models(algo, fname):
  pickleFile = drive_location+algo+"_"+fname+".pickle"
  f = open(pickleFile, 'rb')
  # disable garbage collector
  gc.disable()
  # The protocol version used is detected automatically, so we do not
  # have to specify it.
  data = pickle.load(f)
  # enable garbage collector again
  gc.enable()
  f.close()
  return data

# model

1. nn model
1. keras model

## NN model

In [ ]:
def build_nn_model(algo):
  dataset = load_learned_models(algo, "dataset")
  nnmodel = {}
  input_dim = dataset['X_train_standardized'].shape[1]
  output_dim = dataset['y_train_standardized'].shape[1]
  print('input_dim:', input_dim, 'output_dim:', output_dim)
  model_iter = 0
  rmse = 9999999999999999999999999
  model = tf.keras.Sequential()
  model.add(tf.keras.Input(shape=(input_dim,)))
  while (True):
    if (model_iter > 0):
      # add layer with neurons = rmse*100
      model.add(tf.keras.layers.Dense(int(rmse*100), activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(output_dim, activation='linear'))
    # compile model
    model.compile(loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')
    # fit model
    model.fit(dataset['X_train_standardized'], dataset['y_train_standardized'], epochs=100, verbose=0)
    # evaluate the model
    y_train_pred = model.predict(dataset['X_train_standardized'])
    y_test_pred = model.predict(dataset['X_test_standardized'])
    prev_rmse = rmse
    rmse = mean_squared_error(dataset['y_train_standardized'],y_train_pred,squared=False)
    print(rmse, prev_rmse)
    if (rmse > prev_rmse):
      break
    #nnmodel['model'] = model
    model.save(drive_location+algo+"_nnmodel.h5")
    nnmodel[model_iter] = {}
    nnmodel[model_iter]['y_train'] = dataset['y_train_standardized']
    nnmodel[model_iter]['y_train_pred'] = y_train_pred
    nnmodel[model_iter]['y_test'] = dataset['y_test_standardized']
    nnmodel[model_iter]['y_test_pred'] = y_test_pred
    nnmodel[model_iter]['train_rmse'] = mean_squared_error(dataset['y_train_standardized'],y_train_pred,squared=False)
    nnmodel[model_iter]['train_medae'] = median_absolute_error(dataset['y_train_standardized'],y_train_pred)
    nnmodel[model_iter]['train_explained_var'] = explained_variance_score(dataset['y_train_standardized'],y_train_pred)
    nnmodel[model_iter]['test_rmse'] = mean_squared_error(dataset['y_test_standardized'],y_test_pred,squared=False)
    nnmodel[model_iter]['test_medae'] = median_absolute_error(dataset['y_test_standardized'],y_test_pred)
    nnmodel[model_iter]['test_explained_var'] = explained_variance_score(dataset['y_test_standardized'],y_test_pred)
    nnmodel[model_iter]['y_train_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_train_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized'])*100)
    nnmodel[model_iter]['y_test_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_test_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized'])*100)
    nnmodel[model_iter]['train_medape'] = np.median(nnmodel[model_iter]['y_train_pred_error'])
    nnmodel[model_iter]['test_medape'] = np.median(nnmodel[model_iter]['y_test_pred_error'])
    
    print('Neural Network Iteration {0}'.format(model_iter))
    print('Training Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(nnmodel[model_iter]['train_rmse'], nnmodel[model_iter]['train_medae'], nnmodel[model_iter]['train_explained_var'], nnmodel[model_iter]['train_medape']))
    print('Testing Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(nnmodel[model_iter]['test_rmse'], nnmodel[model_iter]['test_medae'], nnmodel[model_iter]['test_explained_var'], nnmodel[model_iter]['test_medape']))
    print(model.summary())
    model_iter = model_iter + 1
    # remove the output layer from previously built model
    model.pop()


  save_learned_models(nnmodel, algo, "nnmodel")
  print('Final Model--->')
  model1 = tf.keras.models.load_model(drive_location+algo+"_nnmodel.h5")
  model1.summary()


## keras model

In [ ]:
def build_autokeras(algo):
  dataset = load_learned_models(algo, "dataset")
  reg = ak.StructuredDataRegressor(max_trials=3, overwrite=True)
  # Feed the tensorflow Dataset to the regressor.
  reg.fit(dataset['X_train_standardized'], dataset['y_train_standardized'], epochs=100)
  # Predict with the best model.
  y_train_pred = reg.predict(dataset['X_train_standardized'])
  y_test_pred = reg.predict(dataset['X_test_standardized'])

  autokerasmodel = {}
  autokerasmodel['y_train'] = dataset['y_train_standardized']
  autokerasmodel['y_train_pred'] = y_train_pred
  autokerasmodel['y_test'] = dataset['y_test_standardized']
  autokerasmodel['y_test_pred'] = y_test_pred
  autokerasmodel['train_rmse'] = mean_squared_error(dataset['y_train_standardized'],y_train_pred,squared=False)
  autokerasmodel['train_medae'] = median_absolute_error(dataset['y_train_standardized'],y_train_pred)
  autokerasmodel['train_explained_var'] = explained_variance_score(dataset['y_train_standardized'],y_train_pred)
  autokerasmodel['test_rmse'] = mean_squared_error(dataset['y_test_standardized'],y_test_pred,squared=False)
  autokerasmodel['test_medae'] = median_absolute_error(dataset['y_test_standardized'],y_test_pred)
  autokerasmodel['test_explained_var'] = explained_variance_score(dataset['y_test_standardized'],y_test_pred)
  autokerasmodel['y_train_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_train_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized'])*100)
  autokerasmodel['y_test_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_test_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized'])*100)
  autokerasmodel['train_medape'] = np.median(autokerasmodel['y_train_pred_error'])
  autokerasmodel['test_medape'] = np.median(autokerasmodel['y_test_pred_error'])
    
  print('Auto Keras-->')
  print('Training Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(autokerasmodel['train_rmse'], autokerasmodel['train_medae'], autokerasmodel['train_explained_var'], autokerasmodel['train_medape']))
  print('Testing Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(autokerasmodel['test_rmse'], autokerasmodel['test_medae'], autokerasmodel['test_explained_var'], autokerasmodel['test_medape']))

  # Export as a Keras Model.
  model = reg.export_model()
  try:
    model.save(drive_location+algo+"_model_autokeras", save_format="tf")
    loaded_model = load_model(drive_location+algo+"_model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)
    #loaded_model = load_model(drive_location+algo+"_model_autokeras")
    print(loaded_model.predict(tf.expand_dims(dataset['X_test_standardized'], -1)))
  except Exception:
    model.save(drive_location+algo+"_autokeras.h5")

  save_learned_models(autokerasmodel, algo, "autokeras")

# plotting Functions

1. set_pub
1. capture_metric_in_csv
1. plot_nnmodel_vs_autokeras
1. capture_iteration_metric_in_csv
1. plot_iteration_metric


In [ ]:
def set_pub():
    mpl.rcdefaults()
    mpl.rcParams['font.size'] = 14
    #mpl.rcParams['font.weight'] = 'bold'
    mpl.rcParams['xtick.labelsize'] = 'xx-large'
    mpl.rcParams['ytick.labelsize'] = 'xx-large'
    mpl.rcParams['figure.titlesize'] = 'xx-large'
    mpl.rcParams['legend.fontsize'] = 'xx-large'
    mpl.rcParams['axes.titlesize'] = 'xx-large'
    mpl.rcParams['axes.titleweight'] = 'bold'
    mpl.rcParams['axes.labelsize'] = 'xx-large'
    mpl.rcParams['axes.labelweight'] = 'bold'
    mpl.rcParams['lines.linewidth'] = 3
    mpl.rcParams['lines.markersize'] = 12
    print(mpl.rcParams['figure.titlesize'], mpl.rcParams['legend.fontsize'])
    plt.rc('figure', dpi=500)

In [ ]:
def capture_metric_in_csv(benchmark_title, algo_list):
  # First capture data in csv file for all algo
  metric_list = []
  for algo in algo_list:
    nnmodel = load_learned_models(algo, "nnmodel")
    final_nnmodel_key = list(nnmodel.keys())[-1]
    autokerasmodel = load_learned_models(algo, "autokeras")
    metric_dict = {}
    metric_dict['algo'] = algo
    # RMSE
    metric_dict['nnmodel_train_rmse'] = nnmodel[final_nnmodel_key]['train_rmse']
    metric_dict['autokerasmodel_train_rmse'] = autokerasmodel['train_rmse']
    metric_dict['nnmodel_test_rmse'] = nnmodel[final_nnmodel_key]['test_rmse']
    metric_dict['autokerasmodel_test_rmse'] = autokerasmodel['test_rmse']
    # MedAE
    metric_dict['nnmodel_train_medae'] = nnmodel[final_nnmodel_key]['train_medae']
    metric_dict['autokerasmodel_train_medae'] = autokerasmodel['train_medae']
    metric_dict['nnmodel_test_medae'] = nnmodel[final_nnmodel_key]['test_medae']
    metric_dict['autokerasmodel_test_medae'] = autokerasmodel['test_medae']
    # MedAPE
    metric_dict['nnmodel_train_medape'] = nnmodel[final_nnmodel_key]['train_medape']
    metric_dict['autokerasmodel_train_medape'] = autokerasmodel['train_medape']
    metric_dict['nnmodel_test_medape'] = nnmodel[final_nnmodel_key]['test_medape']
    metric_dict['autokerasmodel_test_medape'] = autokerasmodel['test_medape']

    print(metric_dict)
    metric_list.append(metric_dict)
  metric_df = pd.DataFrame.from_dict(metric_list)
  metric_df.to_csv(drive_location+benchmark_title+"_metric.csv", line_terminator='\n', index=False)

In [ ]:
def plot_nnmodel_vs_autokeras(benchmark_title, algo_list):
  capture_metric_in_csv(benchmark_title, algo_list)

  metric_df = pd.read_csv(drive_location+benchmark_title+"_metric.csv")
  y_pos = np.arange(metric_df.shape[0])
  print(y_pos)


  # Plot Train and Test MedAE for nnmodel (our model) and autokeras model 
  colors = ['r','k','b','y']
  fig, axes = plt.subplots(figsize=(len(y_pos)*1.5, 4))
  rect1 = []
  rect2 = []
  rect3 = []
  rect4 = []
  width = 0.2
  rect1 = plt.bar(y_pos, metric_df['nnmodel_train_medae'], width, color=colors[0])
  rect2 = plt.bar(y_pos+width, metric_df['autokerasmodel_train_medae'], width, color=colors[1])
  rect3 = plt.bar(y_pos+2*width, metric_df['nnmodel_test_medae'], width, color=colors[2])
  rect4 = plt.bar(y_pos+3*width, metric_df['autokerasmodel_test_medae'], width, color=colors[3])
  
  plt.xticks(y_pos+1.5*width, metric_df['algo'], rotation=20, ha='right')
  plt.ylabel('MedAE')
  plt.xlabel(benchmark_title)
  #lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  lgd = plt.legend((rect1, rect2, rect3,rect4), ("Our Model Train MedAE", "Autokeras Model Train MedAE", "Our Model Test MedAE", "Autokeras Test MedAE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()
  strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_medae"+plot_file_ext
  if os.path.isfile(strFile):
    os.remove(strFile)
  #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
  fig.savefig(strFile, bbox_inches='tight')


  # Plot Train and Test MedAPE for nnmodel (our model) and autokeras model 
  colors = ['r','k','b','y']
  fig, axes = plt.subplots(figsize=(len(y_pos)*1.5, 4))
  rect1 = []
  rect2 = []
  rect3 = []
  rect4 = []
  width = 0.2
  rect1 = plt.bar(y_pos, metric_df['nnmodel_train_medape'], width, color=colors[0])
  rect2 = plt.bar(y_pos+width, metric_df['autokerasmodel_train_medape'], width, color=colors[1])
  rect3 = plt.bar(y_pos+2*width, metric_df['nnmodel_test_medape'], width, color=colors[2])
  rect4 = plt.bar(y_pos+3*width, metric_df['autokerasmodel_test_medape'], width, color=colors[3])
  
  plt.xticks(y_pos+1.5*width, metric_df['algo'], rotation=20, ha='right')
  plt.ylabel('MedAPE')
  plt.xlabel(benchmark_title)
  #lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  lgd = plt.legend((rect1, rect2, rect3,rect4), ("Our Model Train MedAPE", "Autokeras Model Train MedAPE", "Our Model Test MedAPE", "Autokeras Test MedAPE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()
  strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_medape"+plot_file_ext
  if os.path.isfile(strFile):
    os.remove(strFile)
  #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
  fig.savefig(strFile, bbox_inches='tight')


  # drop RMSE values for SPEC2006 Int Benchmarks due to very high value
  if (benchmark_title == 'SPEC_Int'):
    indexNames = metric_df[metric_df['algo'].isin(['401.bzip2', '456.hmmer', '458.sjeng', '462.libquantum', '473.astar'])].index
    #print(indexNames)
    metric_df = metric_df.drop(indexNames)
    #print(metric_df)
  y_pos = np.arange(metric_df.shape[0])
  print(y_pos)

   # Plot Train and Test RMSE for nnmodel (our model) and autokeras model 
  colors = ['r','k','b','y']
  fig, axes = plt.subplots(figsize=(len(y_pos)*1.5, 4))
  rect1 = []
  rect2 = []
  rect3 = []
  rect4 = []
  width = 0.2
  rect1 = plt.bar(y_pos, metric_df['nnmodel_train_rmse'], width, color=colors[0])
  rect2 = plt.bar(y_pos+width, metric_df['autokerasmodel_train_rmse'], width, color=colors[1])
  rect3 = plt.bar(y_pos+2*width, metric_df['nnmodel_test_rmse'], width, color=colors[2])
  rect4 = plt.bar(y_pos+3*width, metric_df['autokerasmodel_test_rmse'], width, color=colors[3])
  
  plt.xticks(y_pos+1.5*width, metric_df['algo'], rotation=20, ha='right')
  plt.ylabel('RMSE')
  plt.xlabel(benchmark_title)
  #lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  lgd = plt.legend((rect1, rect2, rect3,rect4), ("Our Model Train RMSE", "Autokeras Model Train RMSE", "Our Model Test RMSE", "Autokeras Test RMSE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()
  strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_rmse"+plot_file_ext
  if os.path.isfile(strFile):
    os.remove(strFile)
  #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
  fig.savefig(strFile, bbox_inches='tight')


In [ ]:
def capture_iteration_metric_in_csv(benchmark_title, algo_list):
  # First capture data in csv file for all algo
  metric_list_train_rmse = []
  metric_list_train_medae = []
  metric_list_train_medape = []
  metric_list_test_rmse = []
  metric_list_test_medae = []
  metric_list_test_medape = []
  for algo in algo_list:
    nnmodel = load_learned_models(algo, "nnmodel")
    nnmodel_keys = list(nnmodel.keys())
    print(nnmodel_keys)
    metric_dict_train_rmse = {}
    metric_dict_train_medae = {}
    metric_dict_train_medape = {}
    metric_dict_test_rmse = {}
    metric_dict_test_medae = {}
    metric_dict_test_medape = {}

    metric_dict_train_rmse['algo'] = algo
    metric_dict_train_medae['algo'] = algo
    metric_dict_train_medape['algo'] = algo
    metric_dict_test_rmse['algo'] = algo
    metric_dict_test_medae['algo'] = algo
    metric_dict_test_medape['algo'] = algo
    
    for key in nnmodel_keys:
      # RMSE
      metric_dict_train_rmse[key] = nnmodel[key]['train_rmse']
      metric_dict_test_rmse[key] = nnmodel[key]['test_rmse']
      # MedAE
      metric_dict_train_medae[key] = nnmodel[key]['train_medae']
      metric_dict_test_medae[key] = nnmodel[key]['test_medae']
      # MedAPE
      metric_dict_train_medape[key] = nnmodel[key]['train_medape']
      metric_dict_test_medape[key] = nnmodel[key]['test_medape']

    print(metric_dict_train_rmse)
    metric_list_train_rmse.append(metric_dict_train_rmse)
    metric_df_train_rmse = pd.DataFrame.from_dict(metric_list_train_rmse)
    metric_df_train_rmse.to_csv(drive_location+benchmark_title+"_metric_train_rmse.csv", line_terminator='\n', index=False)

    metric_list_test_rmse.append(metric_dict_test_rmse)
    metric_df_test_rmse = pd.DataFrame.from_dict(metric_list_test_rmse)
    metric_df_test_rmse.to_csv(drive_location+benchmark_title+"_metric_test_rmse.csv", line_terminator='\n', index=False)

    metric_list_train_medae.append(metric_dict_train_medae)
    metric_df_train_medae = pd.DataFrame.from_dict(metric_list_train_medae)
    metric_df_train_medae.to_csv(drive_location+benchmark_title+"_metric_train_medae.csv", line_terminator='\n', index=False)

    metric_list_test_medae.append(metric_dict_test_medae)
    metric_df_test_medae = pd.DataFrame.from_dict(metric_list_test_medae)
    metric_df_test_medae.to_csv(drive_location+benchmark_title+"_metric_test_medae.csv", line_terminator='\n', index=False)

    metric_list_train_medape.append(metric_dict_train_medape)
    metric_df_train_medape = pd.DataFrame.from_dict(metric_list_train_medape)
    metric_df_train_medape.to_csv(drive_location+benchmark_title+"_metric_train_medape.csv", line_terminator='\n', index=False)

    metric_list_test_medape.append(metric_dict_test_medape)
    metric_df_test_medape = pd.DataFrame.from_dict(metric_list_test_medape)
    metric_df_test_medape.to_csv(drive_location+benchmark_title+"_metric_test_medape.csv", line_terminator='\n', index=False) 

In [ ]:
def plot_iteration_metric(benchmark_title, algo_list):
  capture_iteration_metric_in_csv(benchmark_title, algo_list)

  metrics = ['train_rmse','test_rmse','train_medae','test_medae','train_medape','test_medape']

  for metric in metrics:
    metric_df = pd.read_csv(drive_location+benchmark_title+"_metric_"+metric+".csv")

    fig, axes = plt.subplots(1,1)
    for algo_idx, algo in enumerate(metric_df['algo']):
      print(metric_df.values[algo_idx][1:])
      x_pos = np.arange(len(metric_df.values[algo_idx][1:]))
      print(x_pos)
      plt.plot(x_pos, metric_df.values[algo_idx][1:])
    plt.xticks(x_pos)
    plt.ylabel(metric.upper())
    plt.xlabel('Iterations')
    plt.title('Iterations vs '+metric.upper())
    ldg = plt.legend(metric_df['algo'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_metric_"+metric+plot_file_ext
    if os.path.isfile(strFile):
      os.remove(strFile)
    #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

# import Packages

1. pandas
1. categrory_encoders
1. sklearn
1. pickle
1. gc
1. numpy


1. tensorflow
1. autokeras
1. keras-tuner

In [ ]:
import pandas as pd
!pip install category_encoders
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import gc
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, median_absolute_error, explained_variance_score
import numpy as np
import tensorflow as tf

!pip install git+https://github.com/keras-team/keras-tuner.git
!pip install autokeras
import autokeras as ak
from tensorflow.keras.models import load_model

## package for ploating  graph

In [16]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import csv

# drive Location

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
drive_location = r"/content/drive/MyDrive/Colab Notebooks/Internship/"

# AutoML model function call



In [ ]:
# Mantevo Suite
#benchmark = "Mantevo"
#algo="mantevo_miniFE"
#algo_fname="runtimes_final_mantevo_miniFE"
# NPB Suite
#benchmark = "NPB"
#algo = "npb_ep"
#algo_fname="runtimes_final_npb_ep"
#benchmark = "NPB"
#algo = "npb_mg"
#algo_fname="runtimes_final_npb_mg"
# SPEC 2006 Float
#benchmark = "SPEC2006"
#algo = "leslie3d"
#algo_fname = "437.leslie3d"
# SPEC 2006 Int
#benchmark = "SPEC2006"
#algo = "perlbench"
#algo_fname = "400.perlbench"

spec_float_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_float_algo_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]
spec_float_algo_fname_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]

spec_int_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_int_algo_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]
spec_int_algo_fname_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]

npb_other_benchmark_list = ["NPB","NPB","NPB","NPB","OTHER","OTHER","OTHER"]
npb_other_algo_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]
npb_other_algo_fname_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]


# Assign variable with which list is to be processed
benchmark_list = npb_other_benchmark_list
algo_list = npb_other_algo_list
algo_fname_list = npb_other_algo_fname_list
for algo_idx, algo in enumerate(algo_list):
  # Perform train-test data split and scaling
  perform_dataset_ops(benchmark_list[algo_idx], algo_list[algo_idx], algo_fname_list[algo_idx])

  build_nn_model(algo)

  build_autokeras(algo)



# ploatting

In [ ]:
#Plotting

#benchmark_list = ["NPB", "NPB", "SPEC2006", "SPEC2006"]
#algo_list = ["npb_ep", "npb_mg", "leslie3d", "perlbench"]

spec_float_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_float_algo_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]
spec_float_algo_fname_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]
spec_float_benchmark_title = 'SPEC_Float'

spec_int_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_int_algo_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]
spec_int_algo_fname_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]
spec_int_benchmark_title = 'SPEC_Int'

npb_other_benchmark_list = ["NPB","NPB","NPB","NPB","OTHER","OTHER","OTHER"]
npb_other_algo_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]
npb_other_algo_fname_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]
npb_other_benchmark_title = 'NPB_OTHER'

# Assign variable with which list is to be processed
benchmark_list = npb_other_benchmark_list
algo_list = npb_other_algo_list
algo_fname_list = npb_other_algo_fname_list
benchmark_title = npb_other_benchmark_title
plot_file_ext=".eps"

set_pub()
#plot_nnmodel_vs_autokeras(benchmark_title, algo_list)

plot_iteration_metric(benchmark_title, algo_list)

In [14]:
pytorch={};
pytorch['npb_ep']={ 'mean_absolute_error_train_rmse': 0.2172102075529646, 'mean_squared_error_train_rmse': 0.21024337922971625, 'root_mean_squared_error_train_rmse': 0.21024337922971625, 'median_absolute_error_train_rmse': 0.21626906827181736, 'r2_train_rmse': 0.21024337922971625, 'mean_absolute_error_test_rmse': 0.23053108585200344, 'mean_squared_error_test_rmse': 0.22952507573220274, 'root_mean_squared_error_test_rmse': 0.22952507573220274, 'median_absolute_error_test_rmse': 0.22966512422464969, 'r2_test_rmse': 0.22952507573220274, 'mean_absolute_error_train_medae': 0.014212739414404596, 'mean_squared_error_train_medae': 0.0025129794817266276, 'root_mean_squared_error_train_medae': 0.0025129794817266276, 'median_absolute_error_train_medae': 0.014371956838356362, 'r2_train_medae': 0.0025129794817266276, 'mean_absolute_error_test_medae': 0.0326013982668798, 'mean_squared_error_test_medae': 0.026956273395732467, 'root_mean_squared_error_test_medae': 0.026956273395732467, 'median_absolute_error_test_medae': 0.03153451875521723, 'r2_test_medae': 0.026956273395732467, 'mean_absolute_error_train_medape': 1.6074876697931364, 'mean_squared_error_train_medape': 0.2657623918712817, 'root_mean_squared_error_train_medape': 0.2657623918712817, 'median_absolute_error_train_medape': 1.5086079891235533, 'r2_train_medape': 0.2657623918712817, 'mean_absolute_error_test_medape': 4.021811032410793, 'mean_squared_error_test_medape': 2.943498684951407, 'root_mean_squared_error_test_medape': 2.943498684951407, 'median_absolute_error_test_medape': 3.906882216275193, 'r2_test_medape': 2.943498684951407}
pytorch['npb_mg']={ 'mean_absolute_error_train_rmse': 0.1533614331001872, 'mean_squared_error_train_rmse': 0.2050870939329725, 'root_mean_squared_error_train_rmse': 0.2050870939329725, 'median_absolute_error_train_rmse': 0.15727192201701765, 'r2_train_rmse': 0.2050870939329725, 'mean_absolute_error_test_rmse': 0.34533268238128534, 'mean_squared_error_test_rmse': 0.3783803701364594, 'root_mean_squared_error_test_rmse': 0.3783803701364594, 'median_absolute_error_test_rmse': 0.35490141362795263, 'r2_test_rmse': 0.3783803701364594, 'mean_absolute_error_train_medae': 0.031968703157288225, 'mean_squared_error_train_medae': 0.0828150287758751, 'root_mean_squared_error_train_medae': 0.0828150287758751, 'median_absolute_error_train_medae': 0.0387025671160004, 'r2_train_medae': 0.0828150287758751, 'mean_absolute_error_test_medae': 0.04143862391070463, 'mean_squared_error_test_medae': 0.13442187885257575, 'root_mean_squared_error_test_medae': 0.13442187885257575, 'median_absolute_error_test_medae': 0.04696442674460574, 'r2_test_medae': 0.13442187885257575, 'mean_absolute_error_train_medape': 1.7656391946255994, 'mean_squared_error_train_medape': 5.628928348198281, 'root_mean_squared_error_train_medape': 5.628928348198281, 'median_absolute_error_train_medape': 2.411216417891307, 'r2_train_medape': 5.628928348198281, 'mean_absolute_error_test_medape': 2.0966164379777594, 'mean_squared_error_test_medape': 8.116862872681125, 'root_mean_squared_error_test_medape': 8.116862872681125, 'median_absolute_error_test_medape': 2.126568002213985, 'r2_test_medape': 8.116862872681125}
pytorch['npb_sp']={ 'mean_absolute_error_train_rmse': 0.2768356523884434, 'mean_squared_error_train_rmse': 0.2889315904651225, 'root_mean_squared_error_train_rmse': 0.2889315904651225, 'median_absolute_error_train_rmse': 0.42026412514962286, 'r2_train_rmse': 0.2889315904651225, 'mean_absolute_error_test_rmse': 0.4189347924663594, 'mean_squared_error_test_rmse': 0.3647915791808203, 'root_mean_squared_error_test_rmse': 0.3647915791808203, 'median_absolute_error_test_rmse': 0.5500097737786444, 'r2_test_rmse': 0.3647915791808203, 'mean_absolute_error_train_medae': 0.019325980806696563, 'mean_squared_error_train_medae': 0.04682029562128964, 'root_mean_squared_error_train_medae': 0.04682029562128964, 'median_absolute_error_train_medae': 0.06143191299312778, 'r2_train_medae': 0.04682029562128964, 'mean_absolute_error_test_medae': 0.08483974664827759, 'mean_squared_error_test_medae': 0.09776781537073892, 'root_mean_squared_error_test_medae': 0.09776781537073892, 'median_absolute_error_test_medae': 0.08414294794637006, 'r2_test_medae': 0.09776781537073892, 'mean_absolute_error_train_medape': 1.6306362693873773, 'mean_squared_error_train_medape': 3.4539218640420843, 'root_mean_squared_error_train_medape': 3.4539218640420843, 'median_absolute_error_train_medape': 5.251292604504723, 'r2_train_medape': 3.4539218640420843, 'mean_absolute_error_test_medape': 8.548625703933215, 'mean_squared_error_test_medape': 8.473722830040836, 'root_mean_squared_error_test_medape': 8.473722830040836, 'median_absolute_error_test_medape': 8.437605246908795, 'r2_test_medape': 8.473722830040836}
pytorch['npb_sp-mz']={ 'mean_absolute_error_train_rmse': 0.20657939962694138, 'mean_squared_error_train_rmse': 0.2238195680391748, 'root_mean_squared_error_train_rmse': 0.2238195680391748, 'median_absolute_error_train_rmse': 0.3096895143093611, 'r2_train_rmse': 0.2238195680391748, 'mean_absolute_error_test_rmse': 0.17483748891975237, 'mean_squared_error_test_rmse': 0.20786660750086136, 'root_mean_squared_error_test_rmse': 0.20786660750086136, 'median_absolute_error_test_rmse': 0.15430998588287342, 'r2_test_rmse': 0.20786660750086136, 'mean_absolute_error_train_medae': 0.037240839320858324, 'mean_squared_error_train_medae': 0.04244839508884202, 'root_mean_squared_error_train_medae': 0.04244839508884202, 'median_absolute_error_train_medae': 0.0568915654578219, 'r2_train_medae': 0.04244839508884202, 'mean_absolute_error_test_medae': 0.04351131332053154, 'mean_squared_error_test_medae': 0.07295462723644063, 'root_mean_squared_error_test_medae': 0.07295462723644063, 'median_absolute_error_test_medae': 0.07637905388153979, 'r2_test_medae': 0.07295462723644063, 'mean_absolute_error_train_medape': 3.1419792842369603, 'mean_squared_error_train_medape': 4.074949061227586, 'root_mean_squared_error_train_medape': 4.074949061227586, 'median_absolute_error_train_medape': 5.17238849236177, 'r2_train_medape': 4.074949061227586, 'mean_absolute_error_test_medape': 4.096053507728364, 'mean_squared_error_test_medape': 4.809487316393015, 'root_mean_squared_error_test_medape': 4.809487316393015, 'median_absolute_error_test_medape': 6.222324036734376, 'r2_test_medape': 4.809487316393015}
pytorch['matmul']={'mean_absolute_error_train_rmse': 0.02269634528428064, 'mean_squared_error_train_rmse': 0.02269634528428064, 'root_mean_squared_error_train_rmse': 0.02269634528428064, 'median_absolute_error_train_rmse': 0.024027984559360106, 'r2_train_rmse': 0.02269634528428064, 'mean_absolute_error_test_rmse': 0.02636827549187278, 'mean_squared_error_test_rmse': 0.02636827549187278, 'root_mean_squared_error_test_rmse': 0.02636827549187278, 'median_absolute_error_test_rmse': 0.026789599486133082, 'r2_test_rmse': 0.02636827549187278, 'mean_absolute_error_train_medae': 1.3956448191576598e-08, 'mean_squared_error_train_medae': 1.3956448191576598e-08, 'root_mean_squared_error_train_medae': 1.3956448191576598e-08, 'median_absolute_error_train_medae': 0.0019429283535606912, 'r2_train_medae': 1.3956448191576598e-08, 'mean_absolute_error_test_medae': 0.0059416295085835125, 'mean_squared_error_test_medae': 0.0059416295085835125, 'root_mean_squared_error_test_medae': 0.0059416295085835125, 'median_absolute_error_test_medae': 0.007004718153601319, 'r2_test_medae': 0.0059416295085835125, 'mean_absolute_error_train_medape': 2.1475240884977597e-06, 'mean_squared_error_train_medape': 2.1475240884977597e-06, 'root_mean_squared_error_train_medape': 2.1475240884977597e-06, 'median_absolute_error_train_medape': 0.25344916277930735, 'r2_train_medape': 2.1475240884977597e-06, 'mean_absolute_error_test_medape': 0.47958388771683214, 'mean_squared_error_test_medape': 0.47958388771683214, 'root_mean_squared_error_test_medape': 0.47958388771683214, 'median_absolute_error_test_medape': 0.5376725676724321, 'r2_test_medape': 0.47958388771683214}
pytorch['montecarlo']={ 'mean_absolute_error_train_rmse': 0.12549534815443472, 'mean_squared_error_train_rmse': 0.12410701211097498, 'root_mean_squared_error_train_rmse': 0.12410701211097498, 'median_absolute_error_train_rmse': 0.11571318891343052, 'r2_train_rmse': 0.12410701211097498, 'mean_absolute_error_test_rmse': 0.08255630493882789, 'mean_squared_error_test_rmse': 0.081191900456155, 'root_mean_squared_error_test_rmse': 0.081191900456155, 'median_absolute_error_test_rmse': 0.07147756039895481, 'r2_test_rmse': 0.081191900456155, 'mean_absolute_error_train_medae': 0.006224564763839757, 'mean_squared_error_train_medae': 0.006365549453442865, 'root_mean_squared_error_train_medae': 0.006365549453442865, 'median_absolute_error_train_medae': 0.004259081572638934, 'r2_train_medae': 0.006365549453442865, 'mean_absolute_error_test_medae': 0.024003001375104888, 'mean_squared_error_test_medae': 0.02267143547346781, 'root_mean_squared_error_test_medae': 0.02267143547346781, 'median_absolute_error_test_medae': 0.014815899927663079, 'r2_test_medae': 0.02267143547346781, 'mean_absolute_error_train_medape': 2.6742660463580235, 'mean_squared_error_train_medape': 2.5995356921384793, 'root_mean_squared_error_train_medape': 2.5995356921384793, 'median_absolute_error_train_medape': 1.6719775907102932, 'r2_train_medape': 2.5995356921384793, 'mean_absolute_error_test_medape': 7.589608121959377, 'mean_squared_error_test_medape': 7.3913440337767895, 'root_mean_squared_error_test_medape': 7.3913440337767895, 'median_absolute_error_test_medape': 5.7579221181538305, 'r2_test_medape': 7.3913440337767895}
pytorch['quicksort']={ 'mean_absolute_error_train_rmse': 0.06307920358606875, 'mean_squared_error_train_rmse': 0.06205280883757895, 'root_mean_squared_error_train_rmse': 0.06205280883757895, 'median_absolute_error_train_rmse': 0.05707208582383326, 'r2_train_rmse': 0.06205280883757895, 'mean_absolute_error_test_rmse': 0.16844367998237195, 'mean_squared_error_test_rmse': 0.16768770376518455, 'root_mean_squared_error_test_rmse': 0.16768770376518455, 'median_absolute_error_test_rmse': 0.16435127118511514, 'r2_test_rmse': 0.16768770376518455, 'mean_absolute_error_train_medae': 0.00905033488134023, 'mean_squared_error_train_medae': 0.0090752905676319, 'root_mean_squared_error_train_medae': 0.0090752905676319, 'median_absolute_error_train_medae': 0.007129442955647672, 'r2_train_medae': 0.0090752905676319, 'mean_absolute_error_test_medae': 0.015683075168595573, 'mean_squared_error_test_medae': 0.016052653768525382, 'root_mean_squared_error_test_medae': 0.016052653768525382, 'median_absolute_error_test_medae': 0.016078641371118745, 'r2_test_medae': 0.016052653768525382, 'mean_absolute_error_train_medape': 0.416989125518587, 'mean_squared_error_train_medape': 0.4074130143395429, 'root_mean_squared_error_train_medape': 0.4074130143395429, 'median_absolute_error_train_medape': 0.31823155505654377, 'r2_train_medape': 0.4074130143395429, 'mean_absolute_error_test_medape': 0.918947800824341, 'mean_squared_error_test_medape': 0.9048448846068711, 'root_mean_squared_error_test_medape': 0.9048448846068711, 'median_absolute_error_test_medape': 0.8697579896470415, 'r2_test_medape': 0.9048448846068711}

pytorch['SPEC_INT_600.perlbench_s']={'mean_absolute_error_train_rmse': 0.4934457041202531, 'mean_squared_error_train_rmse': 0.29877664565379836, 'root_mean_squared_error_train_rmse': 0.49280051837755245, 'median_absolute_error_train_rmse': 0.49430578979431666, 'r2_train_rmse': 0.29877664565379836, 'mean_absolute_error_test_rmse': 0.6417041967369294, 'mean_squared_error_test_rmse': 0.43565582685035176, 'root_mean_squared_error_test_rmse': 0.6425840761299778, 'median_absolute_error_test_rmse': 0.641006047684147, 'r2_test_rmse': 0.43565582685035176, 'mean_absolute_error_train_medae': 0.08843144915592072, 'mean_squared_error_train_medae': 0.04471507185213175, 'root_mean_squared_error_train_medae': 0.0889889398409398, 'median_absolute_error_train_medae': 0.08919158635955296, 'r2_train_medae': 0.04471507185213175, 'mean_absolute_error_test_medae': 0.10426588880693649, 'mean_squared_error_test_medae': 0.05961348577714187, 'root_mean_squared_error_test_medae': 0.10343353746585504, 'median_absolute_error_test_medae': 0.10964740861380343, 'r2_test_medae': 0.05961348577714187, 'mean_absolute_error_train_medape': 2.1249485746230397, 'mean_squared_error_train_medape': 1.0599514881029402, 'root_mean_squared_error_train_medape': 2.130046954794868, 'median_absolute_error_train_medape': 2.1539436862151455, 'r2_train_medape': 1.0599514881029402, 'mean_absolute_error_test_medape': 2.6446530863181055, 'mean_squared_error_test_medape': 1.3927766826882917, 'root_mean_squared_error_test_medape': 2.647046114613019, 'median_absolute_error_test_medape': 2.6360298096406662, 'r2_test_medape': 1.3927766826882917}
pytorch['SPEC_INT_602.gcc_s']={ 'mean_absolute_error_train_rmse': 0.48839467851063845, 'mean_squared_error_train_rmse': 0.48791988946225495, 'root_mean_squared_error_train_rmse': 0.48791988946225495, 'median_absolute_error_train_rmse': 0.48839467851063845, 'r2_train_rmse': 0.48791988946225495, 'mean_absolute_error_test_rmse': 0.40491822148555184, 'mean_squared_error_test_rmse': 0.40642434590258275, 'root_mean_squared_error_test_rmse': 0.40642434590258275, 'median_absolute_error_test_rmse': 0.40491822148555184, 'r2_test_rmse': 0.40642434590258275, 'mean_absolute_error_train_medae': 0.11947519044517868, 'mean_squared_error_train_medae': 0.11917527582988033, 'root_mean_squared_error_train_medae': 0.11917527582988033, 'median_absolute_error_train_medae': 0.11947519044517868, 'r2_train_medae': 0.11917527582988033, 'mean_absolute_error_test_medae': 0.12414910497531573, 'mean_squared_error_test_medae': 0.12492056789264361, 'root_mean_squared_error_test_medae': 0.12492056789264361, 'median_absolute_error_test_medae': 0.12414910497531573, 'r2_test_medae': 0.12492056789264361, 'mean_absolute_error_train_medape': 2.6399785592533678, 'mean_squared_error_train_medape': 2.658380539750014, 'root_mean_squared_error_train_medape': 2.658380539750014, 'median_absolute_error_train_medape': 2.6399785592533678, 'r2_train_medape': 2.658380539750014, 'mean_absolute_error_test_medape': 2.8084120264402173, 'mean_squared_error_test_medape': 2.795748094877535, 'root_mean_squared_error_test_medape': 2.795748094877535, 'median_absolute_error_test_medape': 2.8084120264402173, 'r2_test_medape': 2.795748094877535}
pytorch['SPEC_INT_605.mcf_s']={ 'mean_absolute_error_train_rmse': 0.3513146522150818, 'mean_squared_error_train_rmse': 0.2666386466307386, 'root_mean_squared_error_train_rmse': 0.35069627271392734, 'median_absolute_error_train_rmse': 0.34985678372271206, 'r2_train_rmse': 0.2666386466307386, 'mean_absolute_error_test_rmse': 0.5035584931259421, 'mean_squared_error_test_rmse': 0.4507426544392861, 'root_mean_squared_error_test_rmse': 0.5049915512516079, 'median_absolute_error_test_rmse': 0.5081676714248001, 'r2_test_rmse': 0.4507426544392861, 'mean_absolute_error_train_medae': 0.09332340043017623, 'mean_squared_error_train_medae': 0.06047582297485665, 'root_mean_squared_error_train_medae': 0.09277768729922647, 'median_absolute_error_train_medae': 0.0886138750434945, 'r2_train_medae': 0.06047582297485665, 'mean_absolute_error_test_medae': 0.10001342166971768, 'mean_squared_error_test_medae': 0.0809027467665074, 'root_mean_squared_error_test_medae': 0.09936143293069935, 'median_absolute_error_test_medae': 0.09955698251592983, 'r2_test_medae': 0.0809027467665074, 'mean_absolute_error_train_medape': 3.0391394686483544, 'mean_squared_error_train_medape': 1.9955604324716343, 'root_mean_squared_error_train_medape': 3.0207786285146367, 'median_absolute_error_train_medape': 3.031603502427025, 'r2_train_medape': 1.9955604324716343, 'mean_absolute_error_test_medape': 3.271824397948471, 'mean_squared_error_test_medape': 2.6587636065204996, 'root_mean_squared_error_test_medape': 3.2349194812144733, 'median_absolute_error_test_medape': 3.25350409067282, 'r2_test_medape': 2.6587636065204996}
pytorch['SPEC_INT_620.omnetpp_s']={ 'mean_absolute_error_train_rmse': 0.44138085804434923, 'mean_squared_error_train_rmse': 0.4441107657356612, 'root_mean_squared_error_train_rmse': 0.4441107657356612, 'median_absolute_error_train_rmse': 0.4441107657356612, 'r2_train_rmse': 0.4441107657356612, 'mean_absolute_error_test_rmse': 0.4538588898248478, 'mean_squared_error_test_rmse': 0.45150573211223, 'root_mean_squared_error_test_rmse': 0.45150573211223, 'median_absolute_error_test_rmse': 0.45150573211223, 'r2_test_rmse': 0.45150573211223, 'mean_absolute_error_train_medae': 0.146824491795141, 'mean_squared_error_train_medae': 0.14977616605438615, 'root_mean_squared_error_train_medae': 0.14977616605438615, 'median_absolute_error_train_medae': 0.14977616605438615, 'r2_train_medae': 0.14977616605438615, 'mean_absolute_error_test_medae': 0.14779663633169088, 'mean_squared_error_test_medae': 0.15349896594214313, 'root_mean_squared_error_test_medae': 0.15349896594214313, 'median_absolute_error_test_medae': 0.15349896594214313, 'r2_test_medae': 0.15349896594214313, 'mean_absolute_error_train_medape': 5.071950183054536, 'mean_squared_error_train_medape': 5.130767534673156, 'root_mean_squared_error_train_medape': 5.130767534673156, 'median_absolute_error_train_medape': 5.130767534673156, 'r2_train_medape': 5.130767534673156, 'mean_absolute_error_test_medape': 5.504833690134116, 'mean_squared_error_test_medape': 5.620301221176782, 'root_mean_squared_error_test_medape': 5.620301221176782, 'median_absolute_error_test_medape': 5.620301221176782, 'r2_test_medape': 5.620301221176782}
pytorch['SPEC_INT_623.xalancbmk_s']={ 'mean_absolute_error_train_rmse': 0.41104141333461414, 'mean_squared_error_train_rmse': 0.28511795450264177, 'root_mean_squared_error_train_rmse': 0.41104141333461414, 'median_absolute_error_train_rmse': 0.41104141333461414, 'r2_train_rmse': 0.41104141333461414, 'mean_absolute_error_test_rmse': 0.5349968422964754, 'mean_squared_error_test_rmse': 0.37785883872871906, 'root_mean_squared_error_test_rmse': 0.5349968422964754, 'median_absolute_error_test_rmse': 0.5349968422964754, 'r2_test_rmse': 0.5349968422964754, 'mean_absolute_error_train_medae': 0.09411523043135694, 'mean_squared_error_train_medae': 0.053111073811894505, 'root_mean_squared_error_train_medae': 0.09411523043135694, 'median_absolute_error_train_medae': 0.09411523043135694, 'r2_train_medae': 0.09411523043135694, 'mean_absolute_error_test_medae': 0.1112430692501532, 'mean_squared_error_test_medae': 0.06499212323036385, 'root_mean_squared_error_test_medae': 0.1112430692501532, 'median_absolute_error_test_medae': 0.1112430692501532, 'r2_test_medae': 0.1112430692501532, 'mean_absolute_error_train_medape': 2.658551605818394, 'mean_squared_error_train_medape': 1.4292392737491804, 'root_mean_squared_error_train_medape': 2.658551605818394, 'median_absolute_error_train_medape': 2.658551605818394, 'r2_train_medape': 2.658551605818394, 'mean_absolute_error_test_medape': 3.2074396045275924, 'mean_squared_error_test_medape': 1.7629208416886093, 'root_mean_squared_error_test_medape': 3.2074396045275924, 'median_absolute_error_test_medape': 3.2074396045275924, 'r2_test_medape': 3.2074396045275924}
pytorch['SPEC_INT_625.x264_s']={ 'mean_absolute_error_train_rmse': 0.8136173723339588, 'mean_squared_error_train_rmse': 0.8136173723339588, 'root_mean_squared_error_train_rmse': 0.8136173723339588, 'median_absolute_error_train_rmse': 0.8169226314794215, 'r2_train_rmse': 0.8136173723339588, 'mean_absolute_error_test_rmse': 0.3083084798992582, 'mean_squared_error_test_rmse': 0.3083084798992582, 'root_mean_squared_error_test_rmse': 0.3083084798992582, 'median_absolute_error_test_rmse': 0.3034295425269214, 'r2_test_rmse': 0.3083084798992582, 'mean_absolute_error_train_medae': 0.058688430849660356, 'mean_squared_error_train_medae': 0.058688430849660356, 'root_mean_squared_error_train_medae': 0.058688430849660356, 'median_absolute_error_train_medae': 0.06350935631164983, 'r2_train_medae': 0.058688430849660356, 'mean_absolute_error_test_medae': 0.06687263291107426, 'mean_squared_error_test_medae': 0.06687263291107426, 'root_mean_squared_error_test_medae': 0.06687263291107426, 'median_absolute_error_test_medae': 0.07332700760852712, 'r2_test_medae': 0.06687263291107426, 'mean_absolute_error_train_medape': 2.8523440286385138, 'mean_squared_error_train_medape': 2.8523440286385138, 'root_mean_squared_error_train_medape': 2.8523440286385138, 'median_absolute_error_train_medape': 2.934862241824369, 'r2_train_medape': 2.8523440286385138, 'mean_absolute_error_test_medape': 3.4135682700046575, 'mean_squared_error_test_medape': 3.4135682700046575, 'root_mean_squared_error_test_medape': 3.4135682700046575, 'median_absolute_error_test_medape': 3.4233398463779974, 'r2_test_medape': 3.4135682700046575}
pytorch['SPEC_INT_631.deepsjeng_s']={'mean_absolute_error_train_rmse': 0.707505279060675, 'mean_squared_error_train_rmse': 0.707505279060675, 'root_mean_squared_error_train_rmse': 0.707505279060675, 'median_absolute_error_train_rmse': 0.7080105843340088, 'r2_train_rmse': 0.707505279060675, 'mean_absolute_error_test_rmse': 0.3980937648469241, 'mean_squared_error_test_rmse': 0.3980937648469241, 'root_mean_squared_error_test_rmse': 0.3980937648469241, 'median_absolute_error_test_rmse': 0.3965470164161434, 'r2_test_rmse': 0.3980937648469241, 'mean_absolute_error_train_medae': 0.07298605987687817, 'mean_squared_error_train_medae': 0.07298605987687817, 'root_mean_squared_error_train_medae': 0.07298605987687817, 'median_absolute_error_train_medae': 0.07230857728455131, 'r2_train_medae': 0.07298605987687817, 'mean_absolute_error_test_medae': 0.08476109220616398, 'mean_squared_error_test_medae': 0.08476109220616398, 'root_mean_squared_error_test_medae': 0.08476109220616398, 'median_absolute_error_test_medae': 0.08385365228240296, 'r2_test_medae': 0.08476109220616398, 'mean_absolute_error_train_medape': 1.8993895253524646, 'mean_squared_error_train_medape': 1.8993895253524646, 'root_mean_squared_error_train_medape': 1.8993895253524646, 'median_absolute_error_train_medape': 1.9047274624403376, 'r2_train_medape': 1.8993895253524646, 'mean_absolute_error_test_medape': 2.270566350660921, 'mean_squared_error_test_medape': 2.270566350660921, 'root_mean_squared_error_test_medape': 2.270566350660921, 'median_absolute_error_test_medape': 2.2612083541980654, 'r2_test_medape': 2.270566350660921}
pytorch['SPEC_INT_641.leela_s']={ 'mean_absolute_error_train_rmse': 0.3859755355251636, 'mean_squared_error_train_rmse': 0.3859755355251636, 'root_mean_squared_error_train_rmse': 0.3859755355251636, 'median_absolute_error_train_rmse': 0.40408396264273216, 'r2_train_rmse': 0.3859755355251636, 'mean_absolute_error_test_rmse': 0.5204037788831257, 'mean_squared_error_test_rmse': 0.5204037788831257, 'root_mean_squared_error_test_rmse': 0.5204037788831257, 'median_absolute_error_test_rmse': 0.509129929564055, 'r2_test_rmse': 0.5204037788831257, 'mean_absolute_error_train_medae': 0.0886646408564643, 'mean_squared_error_train_medae': 0.0886646408564643, 'root_mean_squared_error_train_medae': 0.0886646408564643, 'median_absolute_error_train_medae': 0.09224450879085591, 'r2_train_medae': 0.0886646408564643, 'mean_absolute_error_test_medae': 0.1101033779550672, 'mean_squared_error_test_medae': 0.1101033779550672, 'root_mean_squared_error_test_medae': 0.1101033779550672, 'median_absolute_error_test_medae': 0.10532408735384001, 'r2_test_medae': 0.1101033779550672, 'mean_absolute_error_train_medape': 1.9576142544844466, 'mean_squared_error_train_medape': 1.9576142544844466, 'root_mean_squared_error_train_medape': 1.9576142544844466, 'median_absolute_error_train_medape': 2.055290026346061, 'r2_train_medape': 1.9576142544844466, 'mean_absolute_error_test_medape': 2.3495335856514474, 'mean_squared_error_test_medape': 2.3495335856514474, 'root_mean_squared_error_test_medape': 2.3495335856514474, 'median_absolute_error_test_medape': 2.3286274137535523, 'r2_test_medape': 2.3495335856514474}
pytorch['SPEC_INT_648.exchange2_s']={'mean_absolute_error_train_rmse': 0.3678298224389129, 'mean_squared_error_train_rmse': 0.3678298224389129, 'root_mean_squared_error_train_rmse': 0.3678298224389129, 'median_absolute_error_train_rmse': 0.44203161572067395, 'r2_train_rmse': 0.3678298224389129, 'mean_absolute_error_test_rmse': 0.46501351777652855, 'mean_squared_error_test_rmse': 0.46501351777652855, 'root_mean_squared_error_test_rmse': 0.46501351777652855, 'median_absolute_error_test_rmse': 0.4825158617009521, 'r2_test_rmse': 0.46501351777652855, 'mean_absolute_error_train_medae': 0.0963541891744494, 'mean_squared_error_train_medae': 0.0963541891744494, 'root_mean_squared_error_train_medae': 0.0963541891744494, 'median_absolute_error_train_medae': 0.10185827957564686, 'r2_train_medae': 0.0963541891744494, 'mean_absolute_error_test_medae': 0.11544259495673798, 'mean_squared_error_test_medae': 0.11544259495673798, 'root_mean_squared_error_test_medae': 0.11544259495673798, 'median_absolute_error_test_medae': 0.10162743563320661, 'r2_test_medae': 0.11544259495673798, 'mean_absolute_error_train_medape': 2.878132352318588, 'mean_squared_error_train_medape': 2.878132352318588, 'root_mean_squared_error_train_medape': 2.878132352318588, 'median_absolute_error_train_medape': 3.1143313209302104, 'r2_train_medape': 2.878132352318588, 'mean_absolute_error_test_medape': 3.4043520563425154, 'mean_squared_error_test_medape': 3.4043520563425154, 'root_mean_squared_error_test_medape': 3.4043520563425154, 'median_absolute_error_test_medape': 2.9242824896301602, 'r2_test_medape': 3.4043520563425154}
pytorch['SPEC_INT_657.xz_s']={ 'mean_absolute_error_train_rmse': 0.5203130801490314, 'mean_squared_error_train_rmse': 0.5044362356989412, 'root_mean_squared_error_train_rmse': 0.5044362356989412, 'median_absolute_error_train_rmse': 0.5817688010589483, 'r2_train_rmse': 0.5044362356989412, 'mean_absolute_error_test_rmse': 0.7407017165163381, 'mean_squared_error_test_rmse': 0.7302181281266592, 'root_mean_squared_error_test_rmse': 0.7302181281266592, 'median_absolute_error_test_rmse': 0.7876311342681469, 'r2_test_rmse': 0.7302181281266592, 'mean_absolute_error_train_medae': 0.06334568975164201, 'mean_squared_error_train_medae': 0.06349309574059459, 'root_mean_squared_error_train_medae': 0.06349309574059459, 'median_absolute_error_train_medae': 0.0672505932189178, 'r2_train_medae': 0.06349309574059459, 'mean_absolute_error_test_medae': 0.07165611138706496, 'mean_squared_error_test_medae': 0.07457808749480588, 'root_mean_squared_error_test_medae': 0.07457808749480588, 'median_absolute_error_test_medae': 0.06701427232921109, 'r2_test_medae': 0.07457808749480588, 'mean_absolute_error_train_medape': 2.8581233551132947, 'mean_squared_error_train_medape': 2.937860428135157, 'root_mean_squared_error_train_medape': 2.937860428135157, 'median_absolute_error_train_medape': 3.049763865156008, 'r2_train_medape': 2.937860428135157, 'mean_absolute_error_test_medape': 3.1752727190212426, 'mean_squared_error_test_medape': 3.380827735088885, 'root_mean_squared_error_test_medape': 3.380827735088885, 'median_absolute_error_test_medape': 2.9898804202696345, 'r2_test_medape': 3.380827735088885}
pytorch['SPEC_INT_401.bzip2']={ 'mean_absolute_error_train_rmse': 0.26576631628081504, 'mean_squared_error_train_rmse': 0.26576631628081504, 'root_mean_squared_error_train_rmse': 0.26576631628081504, 'median_absolute_error_train_rmse': 0.26576631628081504, 'r2_train_rmse': 0.26576631628081504, 'mean_absolute_error_test_rmse': 0.3071323294610572, 'mean_squared_error_test_rmse': 0.3071323294610572, 'root_mean_squared_error_test_rmse': 0.3071323294610572, 'median_absolute_error_test_rmse': 0.3071323294610572, 'r2_test_rmse': 0.3071323294610572, 'mean_absolute_error_train_medae': 0.1108142189210739, 'mean_squared_error_train_medae': 0.1108142189210739, 'root_mean_squared_error_train_medae': 0.1108142189210739, 'median_absolute_error_train_medae': 0.1108142189210739, 'r2_train_medae': 0.1108142189210739, 'mean_absolute_error_test_medae': 0.11751241956488312, 'mean_squared_error_test_medae': 0.11751241956488312, 'root_mean_squared_error_test_medae': 0.11751241956488312, 'median_absolute_error_test_medae': 0.11751241956488312, 'r2_test_medae': 0.11751241956488312, 'mean_absolute_error_train_medape': 4.069703594791043, 'mean_squared_error_train_medape': 4.069703594791043, 'root_mean_squared_error_train_medape': 4.069703594791043, 'median_absolute_error_train_medape': 4.069703594791043, 'r2_train_medape': 4.069703594791043, 'mean_absolute_error_test_medape': 4.111951736090456, 'mean_squared_error_test_medape': 4.111951736090456, 'root_mean_squared_error_test_medape': 4.111951736090456, 'median_absolute_error_test_medape': 4.111951736090456, 'r2_test_medape': 4.111951736090456}
pytorch['SPEC_INT_456.hmmer']={ 'mean_absolute_error_train_rmse': 0.34181019958100917, 'mean_squared_error_train_rmse': 0.34181019958100917, 'root_mean_squared_error_train_rmse': 0.34181019958100917, 'median_absolute_error_train_rmse': 0.34181019958100917, 'r2_train_rmse': 0.34181019958100917, 'mean_absolute_error_test_rmse': 0.3505408836281231, 'mean_squared_error_test_rmse': 0.3505408836281231, 'root_mean_squared_error_test_rmse': 0.3505408836281231, 'median_absolute_error_test_rmse': 0.3505408836281231, 'r2_test_rmse': 0.3505408836281231, 'mean_absolute_error_train_medae': 0.08250013622500463, 'mean_squared_error_train_medae': 0.08250013622500463, 'root_mean_squared_error_train_medae': 0.08250013622500463, 'median_absolute_error_train_medae': 0.08250013622500463, 'r2_train_medae': 0.08250013622500463, 'mean_absolute_error_test_medae': 0.07921465458979748, 'mean_squared_error_test_medae': 0.07921465458979748, 'root_mean_squared_error_test_medae': 0.07921465458979748, 'median_absolute_error_test_medae': 0.07921465458979748, 'r2_test_medae': 0.07921465458979748, 'mean_absolute_error_train_medape': 7.565399755720824, 'mean_squared_error_train_medape': 7.565399755720824, 'root_mean_squared_error_train_medape': 7.565399755720824, 'median_absolute_error_train_medape': 7.565399755720824, 'r2_train_medape': 7.565399755720824, 'mean_absolute_error_test_medape': 7.701330297904413, 'mean_squared_error_test_medape': 7.701330297904413, 'root_mean_squared_error_test_medape': 7.701330297904413, 'median_absolute_error_test_medape': 7.701330297904413, 'r2_test_medape': 7.701330297904413}
pytorch['SPEC_INT_458.sjeng']={'mean_absolute_error_train_rmse': 0.24669403856138747, 'mean_squared_error_train_rmse': 0.24669403856138747, 'root_mean_squared_error_train_rmse': 0.24669403856138747, 'median_absolute_error_train_rmse': 0.24669403856138747, 'r2_train_rmse': 0.24669403856138747, 'mean_absolute_error_test_rmse': 0.3447934912136243, 'mean_squared_error_test_rmse': 0.3447934912136243, 'root_mean_squared_error_test_rmse': 0.3447934912136243, 'median_absolute_error_test_rmse': 0.3447934912136243, 'r2_test_rmse': 0.3447934912136243, 'mean_absolute_error_train_medae': 0.099826929886345, 'mean_squared_error_train_medae': 0.099826929886345, 'root_mean_squared_error_train_medae': 0.099826929886345, 'median_absolute_error_train_medae': 0.099826929886345, 'r2_train_medae': 0.099826929886345, 'mean_absolute_error_test_medae': 0.09964781916206966, 'mean_squared_error_test_medae': 0.09964781916206966, 'root_mean_squared_error_test_medae': 0.09964781916206966, 'median_absolute_error_test_medae': 0.09964781916206966, 'r2_test_medae': 0.09964781916206966, 'mean_absolute_error_train_medape': 3.6188547386708314, 'mean_squared_error_train_medape': 3.6188547386708314, 'root_mean_squared_error_train_medape': 3.6188547386708314, 'median_absolute_error_train_medape': 3.6188547386708314, 'r2_train_medape': 3.6188547386708314, 'mean_absolute_error_test_medape': 3.6905943313389318, 'mean_squared_error_test_medape': 3.6905943313389318, 'root_mean_squared_error_test_medape': 3.6905943313389318, 'median_absolute_error_test_medape': 3.6905943313389318, 'r2_test_medape': 3.6905943313389318}
pytorch['SPEC_INT_462.libquantum']={ 'mean_absolute_error_train_rmse': 0.6104357780054849, 'mean_squared_error_train_rmse': 0.6104357780054849, 'root_mean_squared_error_train_rmse': 0.6104357780054849, 'median_absolute_error_train_rmse': 0.6104357780054849, 'r2_train_rmse': 0.6104357780054849, 'mean_absolute_error_test_rmse': 0.5957181259413068, 'mean_squared_error_test_rmse': 0.5957181259413068, 'root_mean_squared_error_test_rmse': 0.5957181259413068, 'median_absolute_error_test_rmse': 0.5957181259413068, 'r2_test_rmse': 0.5957181259413068, 'mean_absolute_error_train_medae': 0.06730148252065443, 'mean_squared_error_train_medae': 0.06730148252065443, 'root_mean_squared_error_train_medae': 0.06730148252065443, 'median_absolute_error_train_medae': 0.06730148252065443, 'r2_train_medae': 0.06730148252065443, 'mean_absolute_error_test_medae': 0.0671925756333398, 'mean_squared_error_test_medae': 0.0671925756333398, 'root_mean_squared_error_test_medae': 0.0671925756333398, 'median_absolute_error_test_medae': 0.0671925756333398, 'r2_test_medae': 0.0671925756333398, 'mean_absolute_error_train_medape': 53.00870324090163, 'mean_squared_error_train_medape': 53.00870324090163, 'root_mean_squared_error_train_medape': 53.00870324090163, 'median_absolute_error_train_medape': 53.00870324090163, 'r2_train_medape': 53.00870324090163, 'mean_absolute_error_test_medape': 55.20010243757976, 'mean_squared_error_test_medape': 55.20010243757976, 'root_mean_squared_error_test_medape': 55.20010243757976, 'median_absolute_error_test_medape': 55.20010243757976, 'r2_test_medape': 55.20010243757976}
pytorch['SPEC_INT_473.astar']={ 'mean_absolute_error_train_rmse': 0.23854868228588774, 'mean_squared_error_train_rmse': 0.23854868228588774, 'root_mean_squared_error_train_rmse': 0.23854868228588774, 'median_absolute_error_train_rmse': 0.23854868228588774, 'r2_train_rmse': 0.23854868228588774, 'mean_absolute_error_test_rmse': 0.2759677510745927, 'mean_squared_error_test_rmse': 0.2759677510745927, 'root_mean_squared_error_test_rmse': 0.2759677510745927, 'median_absolute_error_test_rmse': 0.2759677510745927, 'r2_test_rmse': 0.2759677510745927, 'mean_absolute_error_train_medae': 0.10174038231041493, 'mean_squared_error_train_medae': 0.10174038231041493, 'root_mean_squared_error_train_medae': 0.10174038231041493, 'median_absolute_error_train_medae': 0.10174038231041493, 'r2_train_medae': 0.10174038231041493, 'mean_absolute_error_test_medae': 0.10839248733155826, 'mean_squared_error_test_medae': 0.10839248733155826, 'root_mean_squared_error_test_medae': 0.10839248733155826, 'median_absolute_error_test_medae': 0.10839248733155826, 'r2_test_medae': 0.10839248733155826, 'mean_absolute_error_train_medape': 5.261500209915654, 'mean_squared_error_train_medape': 5.261500209915654, 'root_mean_squared_error_train_medape': 5.261500209915654, 'median_absolute_error_train_medape': 5.261500209915654, 'r2_train_medape': 5.261500209915654, 'mean_absolute_error_test_medape': 5.497064817299542, 'mean_squared_error_test_medape': 5.497064817299542, 'root_mean_squared_error_test_medape': 5.497064817299542, 'median_absolute_error_test_medape': 5.497064817299542, 'r2_test_medape': 5.497064817299542}


pytorch['SPEC_Float_603.bwaves_s']={ 'mean_absolute_error_train_rmse': 0.06654876978144343, 'mean_squared_error_train_rmse': 0.25228030208124036, 'median_absolute_error_train_rmse': 0.25228030208124036, 'r2_train_rmse': 0.25228030208124036, 'mean_absolute_error_test_rmse': 0.1586250119759253, 'mean_squared_error_test_rmse': 0.1973878554679823, 'median_absolute_error_test_rmse': 0.1973878554679823, 'r2_test_rmse': 0.1973878554679823, 'mean_absolute_error_train_medae': 0.008951042278028176, 'mean_squared_error_train_medae': 0.029250132021908643, 'median_absolute_error_train_medae': 0.029250132021908643, 'r2_train_medae': 0.029250132021908643, 'mean_absolute_error_test_medae': 0.019493431546701384, 'mean_squared_error_test_medae': 0.03279772312485413, 'median_absolute_error_test_medae': 0.03279772312485413, 'r2_test_medae': 0.03279772312485413, 'mean_absolute_error_train_medape': 1.143885083550621, 'mean_squared_error_train_medape': 4.183361926491466, 'median_absolute_error_train_medape': 4.183361926491466, 'r2_train_medape': 4.183361926491466, 'mean_absolute_error_test_medape': 2.3010010866279518, 'mean_squared_error_test_medape': 4.65878135484454, 'median_absolute_error_test_medape': 4.65878135484454, 'r2_test_medape': 4.65878135484454}
pytorch['SPEC_Float_607.cactuBSSN_s']={ 'mean_absolute_error_train_rmse': 0.14763450910409198, 'mean_squared_error_train_rmse': 0.7068258314325175, 'median_absolute_error_train_rmse': 0.7343527482421974, 'r2_train_rmse': 0.7068258314325175, 'mean_absolute_error_test_rmse': 0.09478029260574965, 'mean_squared_error_test_rmse': 0.25287612533040044, 'median_absolute_error_test_rmse': 0.27478991209103854, 'r2_test_rmse': 0.25287612533040044, 'mean_absolute_error_train_medae': 0.007950318378422139, 'mean_squared_error_train_medae': 0.030005025108564676, 'median_absolute_error_train_medae': 0.03090317384464658, 'r2_train_medae': 0.030005025108564676, 'mean_absolute_error_test_medae': 0.018292303003255225, 'mean_squared_error_test_medae': 0.03430856931942011, 'median_absolute_error_test_medae': 0.033467397337277355, 'r2_test_medae': 0.03430856931942011, 'mean_absolute_error_train_medape': 1.2466576729405605, 'mean_squared_error_train_medape': 5.067455129059555, 'median_absolute_error_train_medape': 5.505241512986911, 'r2_train_medape': 5.067455129059555, 'mean_absolute_error_test_medape': 2.781552133774454, 'mean_squared_error_test_medape': 5.889877744900394, 'median_absolute_error_test_medape': 5.767260092139411, 'r2_test_medape': 5.889877744900394}
pytorch['SPEC_Float_619.lbm_s']={ 'mean_absolute_error_train_rmse': 0.38094469724278396, 'mean_squared_error_train_rmse': 0.5304173253467663, 'median_absolute_error_train_rmse': 0.48664769206563985, 'r2_train_rmse': 0.5304173253467663, 'mean_absolute_error_test_rmse': 0.3884918928153197, 'mean_squared_error_test_rmse': 0.39058419022234014, 'median_absolute_error_test_rmse': 0.39246082615228867, 'r2_test_rmse': 0.39058419022234014, 'mean_absolute_error_train_medae': 0.025528388217301717, 'mean_squared_error_train_medae': 0.03979609452361696, 'median_absolute_error_train_medae': 0.030110235982274636, 'r2_train_medae': 0.03979609452361696, 'mean_absolute_error_test_medae': 0.027650637658294563, 'mean_squared_error_test_medae': 0.03654163791831627, 'median_absolute_error_test_medae': 0.028106962579934287, 'r2_test_medae': 0.03654163791831627, 'mean_absolute_error_train_medape': 5.375505362629556, 'mean_squared_error_train_medape': 8.782309135211857, 'median_absolute_error_train_medape': 5.932706506604687, 'r2_train_medape': 8.782309135211857, 'mean_absolute_error_test_medape': 5.695743178931744, 'mean_squared_error_test_medape': 8.310870266654234, 'median_absolute_error_test_medape': 5.928920686747181, 'r2_test_medape': 8.310870266654234}
pytorch['SPEC_Float_621.wrf_s']={ 'mean_absolute_error_train_rmse': 0.1335545268017086, 'mean_squared_error_train_rmse': 0.46178555513701625, 'median_absolute_error_train_rmse': 0.47884187909929615, 'r2_train_rmse': 0.46178555513701625, 'mean_absolute_error_test_rmse': 0.1798733671370814, 'mean_squared_error_test_rmse': 0.38170169294550743, 'median_absolute_error_test_rmse': 0.4191960143075493, 'r2_test_rmse': 0.38170169294550743, 'mean_absolute_error_train_medae': 0.014099664144726742, 'mean_squared_error_train_medae': 0.06240615414765005, 'median_absolute_error_train_medae': 0.06706681202137194, 'r2_train_medae': 0.06240615414765005, 'mean_absolute_error_test_medae': 0.031591791429449206, 'mean_squared_error_test_medae': 0.06820129323072485, 'median_absolute_error_test_medae': 0.06652298843150736, 'r2_test_medae': 0.06820129323072485, 'mean_absolute_error_train_medape': 1.2984065766039432, 'mean_squared_error_train_medape': 5.579048811243976, 'median_absolute_error_train_medape': 6.1570573330884635, 'r2_train_medape': 5.579048811243976, 'mean_absolute_error_test_medape': 2.564980401748233, 'mean_squared_error_test_medape': 5.607962338542139, 'median_absolute_error_test_medape': 5.7408214536612565, 'r2_test_medape': 5.607962338542139}
pytorch['SPEC_Float_627.cam4_s']={ 'mean_absolute_error_train_rmse': 0.3505529387379595, 'mean_squared_error_train_rmse': 0.5728443182886459, 'median_absolute_error_train_rmse': 0.5846302719390067, 'r2_train_rmse': 0.5728443182886459, 'mean_absolute_error_test_rmse': 0.15746032457737147, 'mean_squared_error_test_rmse': 0.3282415612583668, 'median_absolute_error_test_rmse': 0.34838787486481504, 'r2_test_rmse': 0.3282415612583668, 'mean_absolute_error_train_medae': 0.008604917235344234, 'mean_squared_error_train_medae': 0.04251080997261103, 'median_absolute_error_train_medae': 0.04027268117765198, 'r2_train_medae': 0.04251080997261103, 'mean_absolute_error_test_medae': 0.02342064567904084, 'mean_squared_error_test_medae': 0.04276821512763082, 'median_absolute_error_test_medae': 0.041169625256986364, 'r2_test_medae': 0.04276821512763082, 'mean_absolute_error_train_medape': 1.2397770360788218, 'mean_squared_error_train_medape': 6.109585249280681, 'median_absolute_error_train_medape': 6.140565315293198, 'r2_train_medape': 6.109585249280681, 'mean_absolute_error_test_medape': 3.4075208654324785, 'mean_squared_error_test_medape': 6.634073404763013, 'median_absolute_error_test_medape': 6.309825744728826, 'r2_test_medape': 6.634073404763013}
pytorch['SPEC_Float_628.pop2_s']={ 'mean_absolute_error_train_rmse': 0.16808370811787124, 'mean_squared_error_train_rmse': 0.6368996384853931, 'median_absolute_error_train_rmse': 0.6904688545263973, 'r2_train_rmse': 0.6368996384853931, 'mean_absolute_error_test_rmse': 0.19515271279101917, 'mean_squared_error_test_rmse': 0.3600862365856865, 'median_absolute_error_test_rmse': 0.42957521269265364, 'r2_test_rmse': 0.3600862365856865, 'mean_absolute_error_train_medae': 0.022569420462816958, 'mean_squared_error_train_medae': 0.08500790439246253, 'median_absolute_error_train_medae': 0.08863549835478826, 'r2_train_medae': 0.08500790439246253, 'mean_absolute_error_test_medae': 0.057046425438072645, 'mean_squared_error_test_medae': 0.09513916861369903, 'median_absolute_error_test_medae': 0.08563401600135159, 'r2_test_medae': 0.09513916861369903, 'mean_absolute_error_train_medape': 1.3100434567191537, 'mean_squared_error_train_medape': 4.843394640940138, 'median_absolute_error_train_medape': 5.268211020883203, 'r2_train_medape': 4.843394640940138, 'mean_absolute_error_test_medape': 3.2449793231846202, 'mean_squared_error_test_medape': 5.305745071071959, 'median_absolute_error_test_medape': 4.911334675629678, 'r2_test_medape': 5.305745071071959}
pytorch['SPEC_Float_638.imagick_s']={'mean_absolute_error_train_rmse': 0.08793341004318656, 'mean_squared_error_train_rmse': 0.6658680354673021, 'median_absolute_error_train_rmse': 0.6658680354673021, 'r2_train_rmse': 0.6658680354673021, 'mean_absolute_error_test_rmse': 0.1127752264921522, 'mean_squared_error_test_rmse': 0.43303198184237607, 'median_absolute_error_test_rmse': 0.43303198184237607, 'r2_test_rmse': 0.43303198184237607, 'mean_absolute_error_train_medae': 0.004883049363495477, 'mean_squared_error_train_medae': 0.022788606995530536, 'median_absolute_error_train_medae': 0.022788606995530536, 'r2_train_medae': 0.022788606995530536, 'mean_absolute_error_test_medae': 0.01264590570377674, 'mean_squared_error_test_medae': 0.02788815830628777, 'median_absolute_error_test_medae': 0.02788815830628777, 'r2_test_medae': 0.02788815830628777, 'mean_absolute_error_train_medape': 1.2899524394994555, 'mean_squared_error_train_medape': 6.9434386287772805, 'median_absolute_error_train_medape': 6.9434386287772805, 'r2_train_medape': 6.9434386287772805, 'mean_absolute_error_test_medape': 3.2257043610192753, 'mean_squared_error_test_medape': 8.262974279805492, 'median_absolute_error_test_medape': 8.262974279805492, 'r2_test_medape': 8.262974279805492}
pytorch['SPEC_Float_644.nab_s']={'mean_absolute_error_train_rmse': 0.1346088004855993, 'mean_squared_error_train_rmse': 0.5201401757478326, 'median_absolute_error_train_rmse': 0.5260406513096357, 'r2_train_rmse': 0.5201401757478326, 'mean_absolute_error_test_rmse': 0.16276324386630464, 'mean_squared_error_test_rmse': 0.444861569041387, 'median_absolute_error_test_rmse': 0.45890897869581204, 'r2_test_rmse': 0.444861569041387, 'mean_absolute_error_train_medae': 0.004779742200374526, 'mean_squared_error_train_medae': 0.03257487756206312, 'median_absolute_error_train_medae': 0.0324265007865982, 'r2_train_medae': 0.03257487756206312, 'mean_absolute_error_test_medae': 0.012653336083161992, 'mean_squared_error_test_medae': 0.03409105563736964, 'median_absolute_error_test_medae': 0.03234151279147657, 'r2_test_medae': 0.03409105563736964, 'mean_absolute_error_train_medape': 0.8794406647947766, 'mean_squared_error_train_medape': 5.8095700577754315, 'median_absolute_error_train_medape': 6.022138915956666, 'r2_train_medape': 5.8095700577754315, 'mean_absolute_error_test_medape': 2.183855709334246, 'mean_squared_error_test_medape': 6.223299955164139, 'median_absolute_error_test_medape': 6.116977534423144, 'r2_test_medape': 6.223299955164139}
pytorch['SPEC_Float_649.fotonik3d_s']={ 'mean_absolute_error_train_rmse': 0.11885740074998234, 'mean_squared_error_train_rmse': 0.3163177164953702, 'median_absolute_error_train_rmse': 0.3184184662622715, 'r2_train_rmse': 0.3163177164953702, 'mean_absolute_error_test_rmse': 0.10412610656168213, 'mean_squared_error_test_rmse': 0.12636046322100347, 'median_absolute_error_test_rmse': 0.1218300898354885, 'r2_test_rmse': 0.12636046322100347, 'mean_absolute_error_train_medae': 0.012357338002410012, 'mean_squared_error_train_medae': 0.03707681057369794, 'median_absolute_error_train_medae': 0.036015853617053395, 'r2_train_medae': 0.03707681057369794, 'mean_absolute_error_test_medae': 0.026539532484805, 'mean_squared_error_test_medae': 0.04334164324871217, 'median_absolute_error_test_medae': 0.04250481621633709, 'r2_test_medae': 0.04334164324871217, 'mean_absolute_error_train_medape': 1.2116072237982738, 'mean_squared_error_train_medape': 3.513937055930648, 'median_absolute_error_train_medape': 3.5679194441616255, 'r2_train_medape': 3.513937055930648, 'mean_absolute_error_test_medape': 2.553275910579319, 'mean_squared_error_test_medape': 4.149325490561546, 'median_absolute_error_test_medape': 4.350902757592221, 'r2_test_medape': 4.149325490561546}
pytorch['SPEC_Float_654.roms_s']={ 'mean_absolute_error_train_rmse': 0.0764594574149435, 'mean_squared_error_train_rmse': 0.302558168767064, 'median_absolute_error_train_rmse': 0.3084834548589351, 'r2_train_rmse': 0.302558168767064, 'mean_absolute_error_test_rmse': 0.07576476900274728, 'mean_squared_error_test_rmse': 0.16660365002525296, 'median_absolute_error_test_rmse': 0.17450974379001474, 'r2_test_rmse': 0.16660365002525296, 'mean_absolute_error_train_medae': 0.006679616988430281, 'mean_squared_error_train_medae': 0.0323821929711916, 'median_absolute_error_train_medae': 0.03234885619940697, 'r2_train_medae': 0.0323821929711916, 'mean_absolute_error_test_medae': 0.015673153557192676, 'mean_squared_error_test_medae': 0.038822846713798695, 'median_absolute_error_test_medae': 0.03689294504606532, 'r2_test_medae': 0.038822846713798695, 'mean_absolute_error_train_medape': 1.3383356540260873, 'mean_squared_error_train_medape': 7.0813732436841015, 'median_absolute_error_train_medape': 7.351090212627394, 'r2_train_medape': 7.0813732436841015, 'mean_absolute_error_test_medape': 3.213071780347677, 'mean_squared_error_test_medape': 8.429360237916494, 'median_absolute_error_test_medape': 8.4502165900394, 'r2_test_medape': 8.429360237916494}
pytorch['SPEC_Float_416.gamess']={ 'mean_absolute_error_train_rmse': 0.10278672133196576, 'mean_squared_error_train_rmse': 0.23679432427769806, 'median_absolute_error_train_rmse': 0.23679432427769806, 'r2_train_rmse': 0.23679432427769806, 'mean_absolute_error_test_rmse': 0.25253024541187014, 'mean_squared_error_test_rmse': 0.35397601959013525, 'median_absolute_error_test_rmse': 0.35397601959013525, 'r2_test_rmse': 0.35397601959013525, 'mean_absolute_error_train_medae': 0.0063544425803973015, 'mean_squared_error_train_medae': 0.10028120832817766, 'median_absolute_error_train_medae': 0.10028120832817766, 'r2_train_medae': 0.10028120832817766, 'mean_absolute_error_test_medae': 0.014807772506309757, 'mean_squared_error_test_medae': 0.10269715102521286, 'median_absolute_error_test_medae': 0.10269715102521286, 'r2_test_medae': 0.10269715102521286, 'mean_absolute_error_train_medape': 0.23854356852054254, 'mean_squared_error_train_medape': 3.80926201447863, 'median_absolute_error_train_medape': 3.80926201447863, 'r2_train_medape': 3.80926201447863, 'mean_absolute_error_test_medape': 0.5156673825881227, 'mean_squared_error_test_medape': 3.8717301982041574, 'median_absolute_error_test_medape': 3.8717301982041574, 'r2_test_medape': 3.8717301982041574}
pytorch['SPEC_Float_433.milc']={ 'mean_absolute_error_train_rmse': 0.1411028463255783, 'mean_squared_error_train_rmse': 0.24035136704869695, 'median_absolute_error_train_rmse': 0.24035136704869695, 'r2_train_rmse': 0.24035136704869695, 'mean_absolute_error_test_rmse': 0.2512385448431954, 'mean_squared_error_test_rmse': 0.32046088338238976, 'median_absolute_error_test_rmse': 0.32046088338238976, 'r2_test_rmse': 0.32046088338238976, 'mean_absolute_error_train_medae': 0.005971665902614998, 'mean_squared_error_train_medae': 0.0700648210644004, 'median_absolute_error_train_medae': 0.0700648210644004, 'r2_train_medae': 0.0700648210644004, 'mean_absolute_error_test_medae': 0.011863951749671253, 'mean_squared_error_test_medae': 0.07588541427896206, 'median_absolute_error_test_medae': 0.07588541427896206, 'r2_test_medae': 0.07588541427896206, 'mean_absolute_error_train_medape': 0.703375275869492, 'mean_squared_error_train_medape': 6.975753847868543, 'median_absolute_error_train_medape': 6.975753847868543, 'r2_train_medape': 6.975753847868543, 'mean_absolute_error_test_medape': 1.3165197775234454, 'mean_squared_error_test_medape': 7.0032195330187665, 'median_absolute_error_test_medape': 7.0032195330187665, 'r2_test_medape': 7.0032195330187665}
pytorch['SPEC_Float_434.zeusmp']={'mean_absolute_error_train_rmse': 0.13748080809847385, 'mean_squared_error_train_rmse': 0.31231596563918246, 'median_absolute_error_train_rmse': 0.31231596563918246, 'r2_train_rmse': 0.31231596563918246, 'mean_absolute_error_test_rmse': 0.24585833194428566, 'mean_squared_error_test_rmse': 0.41624537876215334, 'median_absolute_error_test_rmse': 0.41624537876215334, 'r2_test_rmse': 0.41624537876215334, 'mean_absolute_error_train_medae': 0.00378482711826722, 'mean_squared_error_train_medae': 0.0918691672976944, 'median_absolute_error_train_medae': 0.0918691672976944, 'r2_train_medae': 0.0918691672976944, 'mean_absolute_error_test_medae': 0.009676306269419399, 'mean_squared_error_test_medae': 0.09943516494425703, 'median_absolute_error_test_medae': 0.09943516494425703, 'r2_test_medae': 0.09943516494425703, 'mean_absolute_error_train_medape': 0.6346801423542399, 'mean_squared_error_train_medape': 15.466256103228421, 'median_absolute_error_train_medape': 15.466256103228421, 'r2_train_medape': 15.466256103228421, 'mean_absolute_error_test_medape': 1.7332684654348327, 'mean_squared_error_test_medape': 16.35084669238902, 'median_absolute_error_test_medape': 16.35084669238902, 'r2_test_medape': 16.35084669238902}
pytorch['SPEC_Float_435.gromacs']={ 'mean_absolute_error_train_rmse': 0.13386692175896014, 'mean_squared_error_train_rmse': 0.26395412484556074, 'median_absolute_error_train_rmse': 0.26395412484556074, 'r2_train_rmse': 0.26395412484556074, 'mean_absolute_error_test_rmse': 0.25498409455850196, 'mean_squared_error_test_rmse': 0.3282749888946853, 'median_absolute_error_test_rmse': 0.3282749888946853, 'r2_test_rmse': 0.3282749888946853, 'mean_absolute_error_train_medae': 0.005275007122058528, 'mean_squared_error_train_medae': 0.09482511748197778, 'median_absolute_error_train_medae': 0.09482511748197778, 'r2_train_medae': 0.09482511748197778, 'mean_absolute_error_test_medae': 0.011727460366281112, 'mean_squared_error_test_medae': 0.097990338297297, 'median_absolute_error_test_medae': 0.097990338297297, 'r2_test_medae': 0.097990338297297, 'mean_absolute_error_train_medape': 0.25583507105666475, 'mean_squared_error_train_medape': 4.475838876490545, 'median_absolute_error_train_medape': 4.475838876490545, 'r2_train_medape': 4.475838876490545, 'mean_absolute_error_test_medape': 0.6442058748779789, 'mean_squared_error_test_medape': 4.602996752739781, 'median_absolute_error_test_medape': 4.602996752739781, 'r2_test_medape': 4.602996752739781}
pytorch['SPEC_Float_437.leslie3d']={ 'mean_absolute_error_train_rmse': 0.13713027095695632, 'mean_squared_error_train_rmse': 0.34571835695348635, 'median_absolute_error_train_rmse': 0.34571835695348635, 'r2_train_rmse': 0.34571835695348635, 'mean_absolute_error_test_rmse': 0.1690182167708209, 'mean_squared_error_test_rmse': 0.3995749175220076, 'median_absolute_error_test_rmse': 0.3995749175220076, 'r2_test_rmse': 0.3995749175220076, 'mean_absolute_error_train_medae': 0.005150766540048313, 'mean_squared_error_train_medae': 0.08931005387809382, 'median_absolute_error_train_medae': 0.08931005387809382, 'r2_train_medae': 0.08931005387809382, 'mean_absolute_error_test_medae': 0.012785375067120963, 'mean_squared_error_test_medae': 0.08775958202535494, 'median_absolute_error_test_medae': 0.08775958202535494, 'r2_test_medae': 0.08775958202535494, 'mean_absolute_error_train_medape': 1.3425180294324601, 'mean_squared_error_train_medape': 17.091147372066537, 'median_absolute_error_train_medape': 17.091147372066537, 'r2_train_medape': 17.091147372066537, 'mean_absolute_error_test_medape': 2.589962757675476, 'mean_squared_error_test_medape': 14.5789396158703, 'median_absolute_error_test_medape': 14.5789396158703, 'r2_test_medape': 14.5789396158703}
pytorch['SPEC_Float_447.dealII']={ 'mean_absolute_error_train_rmse': 0.1480129452595243, 'mean_squared_error_train_rmse': 0.32329570828538784, 'median_absolute_error_train_rmse': 0.32329570828538784, 'r2_train_rmse': 0.32329570828538784, 'mean_absolute_error_test_rmse': 0.35016011945749176, 'mean_squared_error_test_rmse': 0.4358151120483469, 'median_absolute_error_test_rmse': 0.4358151120483469, 'r2_test_rmse': 0.4358151120483469, 'mean_absolute_error_train_medae': 0.005171683836466184, 'mean_squared_error_train_medae': 0.10010320558590366, 'median_absolute_error_train_medae': 0.10010320558590366, 'r2_train_medae': 0.10010320558590366, 'mean_absolute_error_test_medae': 0.01634192851346064, 'mean_squared_error_test_medae': 0.1061609064466533, 'median_absolute_error_test_medae': 0.1061609064466533, 'r2_test_medae': 0.1061609064466533, 'mean_absolute_error_train_medape': 0.27161425149006496, 'mean_squared_error_train_medape': 4.909534173173265, 'median_absolute_error_train_medape': 4.909534173173265, 'r2_train_medape': 4.909534173173265, 'mean_absolute_error_test_medape': 0.7410196788078567, 'mean_squared_error_test_medape': 5.161807794327973, 'median_absolute_error_test_medape': 5.161807794327973, 'r2_test_medape': 5.161807794327973}
pytorch['SPEC_Float_450.soplex']={ 'mean_absolute_error_train_rmse': 0.06749711841121427, 'mean_squared_error_train_rmse': 0.19466002696366, 'median_absolute_error_train_rmse': 0.19466002696366, 'r2_train_rmse': 0.19466002696366, 'mean_absolute_error_test_rmse': 0.1306715211531424, 'mean_squared_error_test_rmse': 0.25055907720945414, 'median_absolute_error_test_rmse': 0.25055907720945414, 'r2_test_rmse': 0.25055907720945414, 'mean_absolute_error_train_medae': 0.00788630255702838, 'mean_squared_error_train_medae': 0.09236554929170121, 'median_absolute_error_train_medae': 0.09236554929170121, 'r2_train_medae': 0.09236554929170121, 'mean_absolute_error_test_medae': 0.01521318437325625, 'mean_squared_error_test_medae': 0.09954832461694732, 'median_absolute_error_test_medae': 0.09954832461694732, 'r2_test_medae': 0.09954832461694732, 'mean_absolute_error_train_medape': 0.45481964719994916, 'mean_squared_error_train_medape': 4.924392926967865, 'median_absolute_error_train_medape': 4.924392926967865, 'r2_train_medape': 4.924392926967865, 'mean_absolute_error_test_medape': 0.9350276154288564, 'mean_squared_error_test_medape': 5.340072402006295, 'median_absolute_error_test_medape': 5.340072402006295, 'r2_test_medape': 5.340072402006295}
pytorch['SPEC_Float_453.povray']={ 'mean_absolute_error_train_rmse': 0.06824072515079894, 'mean_squared_error_train_rmse': 0.22443860612509306, 'median_absolute_error_train_rmse': 0.22443860612509306, 'r2_train_rmse': 0.22443860612509306, 'mean_absolute_error_test_rmse': 0.19184918608671891, 'mean_squared_error_test_rmse': 0.32724615078121805, 'median_absolute_error_test_rmse': 0.32724615078121805, 'r2_test_rmse': 0.32724615078121805, 'mean_absolute_error_train_medae': 0.005022094111701125, 'mean_squared_error_train_medae': 0.10002070529780915, 'median_absolute_error_train_medae': 0.10002070529780915, 'r2_train_medae': 0.10002070529780915, 'mean_absolute_error_test_medae': 0.009498281943635986, 'mean_squared_error_test_medae': 0.10511545535337563, 'median_absolute_error_test_medae': 0.10511545535337563, 'r2_test_medae': 0.10511545535337563, 'mean_absolute_error_train_medape': 0.2365130956132055, 'mean_squared_error_train_medape': 4.69880689053869, 'median_absolute_error_train_medape': 4.69880689053869, 'r2_train_medape': 4.69880689053869, 'mean_absolute_error_test_medape': 0.4867215567807728, 'mean_squared_error_test_medape': 4.935915138151879, 'median_absolute_error_test_medape': 4.935915138151879, 'r2_test_medape': 4.935915138151879}
pytorch['SPEC_Float_454.calculix']={ 'mean_absolute_error_train_rmse': 0.1256206919810351, 'mean_squared_error_train_rmse': 0.26635648076455964, 'median_absolute_error_train_rmse': 0.26635648076455964, 'r2_train_rmse': 0.26635648076455964, 'mean_absolute_error_test_rmse': 0.24716544717921393, 'mean_squared_error_test_rmse': 0.3696165653723577, 'median_absolute_error_test_rmse': 0.3696165653723577, 'r2_test_rmse': 0.3696165653723577, 'mean_absolute_error_train_medae': 0.004785473454153921, 'mean_squared_error_train_medae': 0.09154388394437973, 'median_absolute_error_train_medae': 0.09154388394437973, 'r2_train_medae': 0.09154388394437973, 'mean_absolute_error_test_medae': 0.00916749970434251, 'mean_squared_error_test_medae': 0.09590353208963906, 'median_absolute_error_test_medae': 0.09590353208963906, 'r2_test_medae': 0.09590353208963906, 'mean_absolute_error_train_medape': 0.2995749696161664, 'mean_squared_error_train_medape': 4.557051607325871, 'median_absolute_error_train_medape': 4.557051607325871, 'r2_train_medape': 4.557051607325871, 'mean_absolute_error_test_medape': 0.610775160645717, 'mean_squared_error_test_medape': 4.787136629075767, 'median_absolute_error_test_medape': 4.787136629075767, 'r2_test_medape': 4.787136629075767}


In [17]:
save_dict_to_csv(pytorch,'comparisonAll_pytorch')

In [ ]:
# for key,val in tenp.items():
#   if key not in algos:
#     algos[key]=val
#   else:
#     for key2,val2  in val.items():
#       if key2 not in algos[key]:
#         algos[key][key2]=val2
#       else:
#         algos[key][key2]=val2